In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd ..

/Users/purzelrakete/src/github.com/feldberlin/deepspeech


In [2]:
import os

import numpy as np
import torch
from torch.nn import functional as F
import matplotlib.pyplot as plt
import IPython.display as ipd
import wandb

from deepspeech import model, train, datasets, utils, viz, predict

/Users/purzelrakete/src/github.com/feldberlin/deepspeech/.venv/lib/python3.7/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


## Train Deepspeech

In [3]:
dry = True
if dry:
    os.environ['WANDB_MODE'] = 'dryrun'

In [4]:
p = model.HParams(graphemes=datasets.YESNO_GRAPHEMES)
tp = train.HParams(max_epochs=100, batch_size=4, learning_rate=0.00613, grad_norm_clip=200, num_workers=0)
ds = datasets.YesNo(p)
m = model.DeepSpeech(p)
trainset, testset = datasets.splits(ds, p)

# train
t = train.Trainer(m, trainset, testset, tp)

In [ ]:
t.train()

/Users/purzelrakete/src/github.com/feldberlin/deepspeech/.venv/lib/python3.7/site-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")
wandb: Offline run mode, not syncing to the cloud.
wandb: W&B syncing is set to `offline` in this directory.  Run `wandb online` to enable cloud syncing.
wandb: WARNING Symlinked 1 file into the W&B run directory, call wandb.save again to sync new files.
  0%|          | 0/12 [00:00<?, ?it/s]/Users/purzelrakete/src/github.com/feldberlin/deepspeech/.venv/lib/python3.7/site-packages/torchaudio/compliance/kaldi.py:574: UserWarning: The function torch.rfft is deprecated and will be removed in a future PyTorch release. Use the new torch.fft module functions, instead, by importing torch.fft and calling torch.fft.fft or torch.fft.rfft. (Triggered internally at  ../aten/src/ATen/native/S

In [ ]:
viz.ctc_batch(m, [testset[i] for i in range(8)], p)

In [ ]:
yhat, loss = predict.predict_batch(m, [testset[i] for i in range(8)], p)
print(predict.decode_argmax(yhat, p))

In [ ]:
testset[0][1]